In [788]:
from pathlib import Path

import numpy as np
import pypulseq as pp
from bmctool.utils.seq.write import write_seq

import matplotlib.pyplot as plt

In [789]:
try:
    seqid = Path(__file__).stem + "_python"
    folder = Path(__file__).parent
except NameError:
    seqid = "default_seqid"
    folder = Path(".")

In [790]:
# general settings
AUTHOR = "DANIEL MIKSCH"
FLAG_PLOT_SEQUENCE = False  # plot preparation block?
FLAG_CHECK_TIMING = True  # perform a timing check at the end of the sequence?
FLAG_POST_PREP_SPOIL = True  # add spoiler after preparation block?

# sequence definitions
defs: dict = {}
defs["a"] = 90 # a in degrees
defs["b0"] = 17  # B0 [T]
defs["n_trains"] = 1  # number of pulses  #
defs["n_ETM"] = 2

defs["tmix"] = 2.17e-3  # time of exchange between pools (10ms) [s]
defs["trec"] = 3  # recovery time [s]
defs["tETM"] = 0.9e-3 # intermodule delay between rf pulse and gradient
defs["tRead"] = 0.92e-3 # delay between read rf pulse and gradient

# defs["trec_m0"] = 12  # recovery time before M0 [s]
defs["m0_offset"] = -6.1  # m0 offset [ppm]
defs["offsets_ppm"] = np.append(defs["m0_offset"], np.linspace(-6, 6, 21))

# defs["num_meas"] = defs["offsets_ppm"].size  # number of repetition -> Reverse

defs["num_meas"] = defs["offsets_ppm"].size  # number of repetition
# defs["tsat"] = defs["tp"]  # saturation time [s]
defs["seq_id_string"] = seqid  # unique seq id
# defs["spoiling"] = "1" if FLAG_POST_PREP_SPOIL else "0"

seq_filename = defs["seq_id_string"] + ".seq"

# defs["tp"] = 500e-3  # pulse duration [s]
# defs["b1pa"] = 1.174

In [791]:
sys = pp.Opts(
    max_grad=125,
    grad_unit="mT/m",
    max_slew=1000,
    slew_unit="T/m/s",
    rf_ringdown_time=0,
    rf_dead_time=0,
    rf_raster_time=1e-6,
    gamma=42576400,
)

GAMMA_HZ = sys.gamma * 1e-6
defs["freq"] = defs["b0"] * GAMMA_HZ  # Larmor frequency [Hz]

In [792]:
# ===========
# PREPARATION
# ===========

# spoiler
spoil_amp = 0.8 * sys.max_grad  # Hz/m
rise_time = 0e-3  # spoiler rise time in seconds
spoil_dur = 400e-6  # complete spoiler duration in seconds
gmix_duration = 2e-3

gx_spoil, gy_spoil, gz_spoil = [ 
    pp.make_trapezoid(channel=c, 
                      system=sys, 
                      amplitude=spoil_amp,
                      duration=spoil_dur, 
                      rise_time=rise_time)
    for c in ["x", "y", "z"]
]

gx_mix, gy_mix, gz_mix = [ 
    pp.make_trapezoid(channel=c, 
                      system=sys,
                      amplitude=spoil_amp,
                      duration=gmix_duration, 
                      rise_time=rise_time)
    for c in ["x", "y", "z"]
]

# rf pulse
flip_angle = np.radians(defs['a'])
t_pulse = 1.83e-3
t_pulse_read = 3.6e-3
# flip_angle_sat = defs["b1pa"] * GAMMA_HZ * 2 * np.pi * defs["tp"]
# bandwidth_read = 

rf_pulse = pp.make_gauss_pulse(
    flip_angle=flip_angle,
    system=sys,
    bandwidth=1.5e3,         # Bandbreite in Hz
    center_pos=0.5,
    freq_offset=0,
    phase_offset=0,
    return_gz=False               # Slice-Selektionsgradienten zurückgeben
)

rf_read = pp.make_gauss_pulse(
    flip_angle=flip_angle,
    system=sys,
    duration=t_pulse_read,         # Bandbreite in Hz
    center_pos=0.5,
    freq_offset=0,
    phase_offset=0,
    return_gz=False               # Slice-Selektionsgradienten zurückgeben
)


#pseudo adc
pseudo_adc = pp.make_adc(num_samples=1, duration=1e-3)

# delays
tmix_delay = pp.make_delay(defs["tmix"])
trec_delay = pp.make_delay(defs["trec"])
delay_ETM = pp.make_delay(defs["tETM"])
delay_read = pp.make_delay(defs["tRead"])



In [793]:
seq = pp.Sequence()

offsets_hz = defs["offsets_ppm"] * defs["freq"]  # convert from ppm to Hz

In [794]:
for m, offset in enumerate(offsets_hz):
    # print progress/offset
    print(f"#{m + 1} / {len(offsets_hz)} : offset {offset / defs['freq']:.2f} ppm ({offset:.3f} Hz)")

    rf_pulse.freq_offset = offset

    for i in range(defs["n_ETM"]):
        if i == 0:
            seq.add_block(tmix_delay)
        seq.add_block(rf_pulse)
        seq.add_block(delay_ETM)
        seq.add_block(gz_spoil)
        seq.add_block(delay_ETM)
        seq.add_block(rf_pulse)
        seq.add_block(tmix_delay)
        seq.add_block(gz_mix)
        seq.add_block(tmix_delay)


        # add pseudo ADC event
    # seq.add_block(tmix_delay)
    rf_read.freq_offset = 0
    seq.add_block(rf_read)
    seq.add_block(delay_read)
    seq.add_block(gz_spoil)
    seq.add_block(delay_read)
    seq.add_block(pseudo_adc)
    seq.add_block(trec_delay)

if FLAG_CHECK_TIMING:
    ok, error_report = seq.check_timing()
    if ok:
        print("\nTiming check passed successfully")
    else:
        print("\nTiming check failed! Error listing follows\n")
        print(error_report)

#1 / 22 : offset -6.10 ppm (-4415.173 Hz)
#2 / 22 : offset -6.00 ppm (-4342.793 Hz)
#3 / 22 : offset -5.40 ppm (-3908.514 Hz)
#4 / 22 : offset -4.80 ppm (-3474.234 Hz)
#5 / 22 : offset -4.20 ppm (-3039.955 Hz)
#6 / 22 : offset -3.60 ppm (-2605.676 Hz)
#7 / 22 : offset -3.00 ppm (-2171.396 Hz)
#8 / 22 : offset -2.40 ppm (-1737.117 Hz)
#9 / 22 : offset -1.80 ppm (-1302.838 Hz)
#10 / 22 : offset -1.20 ppm (-868.559 Hz)
#11 / 22 : offset -0.60 ppm (-434.279 Hz)
#12 / 22 : offset 0.00 ppm (0.000 Hz)
#13 / 22 : offset 0.60 ppm (434.279 Hz)
#14 / 22 : offset 1.20 ppm (868.559 Hz)
#15 / 22 : offset 1.80 ppm (1302.838 Hz)
#16 / 22 : offset 2.40 ppm (1737.117 Hz)
#17 / 22 : offset 3.00 ppm (2171.396 Hz)
#18 / 22 : offset 3.60 ppm (2605.676 Hz)
#19 / 22 : offset 4.20 ppm (3039.955 Hz)
#20 / 22 : offset 4.80 ppm (3474.234 Hz)
#21 / 22 : offset 5.40 ppm (3908.514 Hz)
#22 / 22 : offset 6.00 ppm (4342.793 Hz)

Timing check passed successfully


In [795]:
if FLAG_PLOT_SEQUENCE:
    seq.plot(time_range=[6.08, 6.13]) #time_range=[0.00, .03]

In [796]:
write_seq(seq=seq, seq_defs=defs, filename=folder / seq_filename, author=AUTHOR, use_matlab_names=True)

In [797]:
pp.calc_rf_bandwidth(rf_read)

array([1040])